Reference FaceAlignment Model
- https://paperswithcode.com/task/face-alignment
- https://github.com/zhenglinzhou/star (STAR Loss: Reducing Semantic Ambiguity in Facial Landmark Detection)
- https://github.com/huangyangyu/ADNet


| Dataset      | Best Model           | Paper | Code  | Compare |
|--------------|----------------------|-------|-------|---------|
| 300W         | STAR                 | 📄    | 🔗   |         |
| WFLW         | LDEQ                 | 📄    | 🔗   |         |
| COFW         | SH-FAN               | 📄    | 🔗   |         |
| AFLW-19      | FaRL-B (epoch 16)    | 📄    | 🔗   |         |
| AFLW2000-3D  | MNN+OR (reannotated) | 📄    | 🔗   |         |




# ***0. CONNECT GOOGLE DRIVE***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ***1. WFLW DATASET AND STRUCTURE DATA FOR TRAINING***
https://wywu.github.io/projects/LAB/WFLW.html

https://datasets.activeloop.ai/docs/ml/datasets/wflw-dataset/


```
# the dataset directory:
|-- ${image_dir}
   |-- WFLW
      | -- WFLW_images

|-- ${annot_dir}
   |-- WFLW
      |-- train.tsv, test.tsv
```

https://github.com/ZhenglinZhou/STAR/blob/master/conf/alignment.py

Quan sát cách data_definition trong file alignment.py
```python
# /conf/alignment.py
        # COFW
        if self.data_definition == "COFW":
            self.edge_info = (
                (True, (0, 4, 2, 5)), # RightEyebrow
                (True, (1, 6, 3, 7)), # LeftEyebrow
                ...
            )
            self.nme_left_index = 16 # pupils
            self.nme_right_index = 17 # pupils
            ...
        # 300W
        elif self.data_definition == "300W":
            self.edge_info = ...
                
        # WFLW
        elif self.data_definition == "WFLW":     
  ```

In [ ]:
# Let's read the provided metadata file to understand its structure
metadata_path = '/content/train.tsv'

# Since we're dealing with a TSV file, we'll use pandas to read it
import pandas as pd

# Read the TSV file into a DataFrame
metadata_df = pd.read_csv(metadata_path, delimiter='\t')

# Display the first few rows of the dataframe to understand its structure
metadata_df.head()


In [ ]:
!pip install deeplake
import deeplake
ds = deeplake.load("hub://activeloop/wflw-train")

In [ ]:
import deeplake
ds_train = deeplake.load("hub://activeloop/wflw-train")[:10]
print(ds_train)
img_list_train = ds_train['images'],
anno_list_train = ds_train['boxes']
print(img_list_train)
print(anno_list_train)

import numpy as np
import matplotlib.pyplot as plt
# Xem qua một số hình ảnh từ tập huấn luyện
i=0
image = ds_train['images'][i]  # Tensor(key='images', index=Index([0]))
print(image)
# In hình dạng và kiểu dữ liệu của ảnh
print("Shape of the image:", image.shape) # Shape of the image: (295, 285, 3)
print("Data type of the image:", image.dtype) # Data type of the image: uint8

image = ds_train['images'][i].numpy()  # Chuyển tensor sang NumPy array
print(image) # Numpy array
print("Shape of the image:", image.shape) # Shape of the image: (295, 285, 3)
print("Data type of the image:", image.dtype) # Data type of the image: uint8

# Hiển thị ảnh
plt.imshow(image)
plt.title(f"Image {i}")
plt.show()

# In thông tin cấu trúc của bounding box
box = ds_train['boxes'][i].numpy()  # Lấy thông tin bounding box và chuyển sang NumPy array
print(f"Box shape for image {i}:", box.shape)
print(f"First bounding box for image {i} (if multiple):", box[0])



Để chuyển đổi dữ liệu từ tập dữ liệu WFLW (được lưu trữ trên DeepLake) sang định dạng TSV (.tsv) giống như mẫu bạn đã cung cấp, bạn cần thực hiện một số bước xử lý dữ liệu. Dưới đây là quy trình cơ bản:

### Bước 1: Truy Cập và Lấy Dữ Liệu
- Đầu tiên, truy cập tập dữ liệu WFLW từ DeepLake.
- Lấy dữ liệu cho các tensors cần thiết như `images`, `boxes`, `keypoints`, và các thuộc tính khác (`blurs`, `expressions`, `illuminations`, `makeups`, `occlusions`, `poses`).

### Bước 2: Xử Lý Dữ Liệu
- **Xử lý Ảnh và Đường dẫn:** Đối với mỗi ảnh, bạn cần lưu đường dẫn tương ứng hoặc tạo một đường dẫn giả lập nếu đang làm việc với dữ liệu ảnh trực tiếp.
- **Xử lý Điểm mốc (Keypoints):** Chuyển đổi dữ liệu keypoints (điểm mốc) thành chuỗi tọa độ. Mỗi điểm mốc sẽ có hai giá trị tọa độ x, y.
- **Xử lý Hộp Giới hạn (Bounding Box):** Trích xuất tọa độ của hộp giới hạn (nếu có) cho mỗi ảnh. Trong trường hợp của bạn, có vẻ như chỉ có ba giá trị, vì vậy cần xác định cách chúng được biểu diễn và xử lý chúng cho phù hợp.
- **Xử lý Thuộc Tính:** Chuyển đổi các giá trị thuộc tính như `occlusion`, `pose`, `make-up`, `illumination`, `blur`, và `expression` thành định dạng số (thường là 0 hoặc 1).

### Bước 3: Ghi ra File TSV
- Kết hợp tất cả các thông tin đã xử lý thành một dòng cho mỗi hình ảnh. Mỗi dòng sẽ bao gồm đường dẫn ảnh, chuỗi tọa độ điểm mốc, tọa độ hộp giới hạn, và giá trị của các thuộc tính.
- Ghi tất cả các dòng vào một file TSV.



# ***1' Load Dataset form Kaggle***
- Sau khi processing, tôi push dataset zip file to Kaggle
https://www.kaggle.com/datasets/cngonngc/facealign-wflwds-adnetstarloss-proccolab-upkaggle

# ***1''. Load EVALUATION Dataset - EDA - Processing Yolov5***
- FOR EVALUATION: CELEBAdataset

# ***2. Model***


Quan sát argparse trong main.py:
https://github.com/ZhenglinZhou/STAR/blob/master/main.py

```main.py python
# main.py

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Entry Fuction")
    parser.add_argument("--mode", type=str, default="train", choices=["train", "test"], help="train or test")
    parser.add_argument("--config_name", type=str, default="alignment", choices=["alignment"], help="set configure file name")
    parser.add_argument("--pretrained_weight", type=str, default=None, help="set pretrained model file name, if ignored then train the network without pretrain model")
    parser.add_argument("--work_dir", type=str, default="./", help="the directory of workspace")
    #parser.add_argument("--device_ids", type=str, default="-1", help="set device ids, -1 means use cpu device, >= 0 means use gpu device")
    #parser.add_argument('--device_ids', type=str, default="0", help="set device ids, -1 means use cpu device, >= 0 means use gpu device")
    parser.add_argument('--device_ids', type=str, default="0,1,2,3", help="set device ids, -1 means use cpu device, >= 0 means use gpu device")
    args = parser.parse_args()



```

In [ ]:
%cd /content/drive/MyDrive/colab
!git clone https://github.com/zhenglinzhou/star
# ModuleNotFoundError: No module named 'tensorboardX'
# Bị lỗi khi pip nên ko pip nữa  !pip install -r requirements.txt


## ***2.1 Wandb, Note Bug in training processing***

## ***2.2 PreTraining and Debug***

## 2.3 SỬA ĐỔI MÃ NGUỒN and Use for Training
Sửa mã fork từ STAR, -> setup ở git cá nhân, load lại xài cho nhanh.
- Fix Bug requirements.txt
- Fix Bug`evalues, evectors = torch.linalg.eigh(covars, UPLO='L') # Doan Ngoc Cuong fix`
trong các file `# !grep -rnw './' -e 'symeig'  # fix bug `

Setup Wandb:
- Setup wandb.init() and wandb.finish() trong main.py
- Log training metrics trong lib/utility.py
- Log valid metrics và save best model trong trainer.py
- Thêm INFO lưu trữ save model trong lib/utility.py
- Add args.pretrained_weights vào main.py, sửa cách save_model with `net` thay vì `best_net` ở file `trainer.py`(best model save torch được log artifacts và lastest model được save torch ko log artifacts)

## 2.4 INFER - PREDECTION - SMALL REAL TESTING

In [ ]:
# %cd /content/drive/MyDrive/colab
!git clone https://github.com/DoanNgocCuongBKEGNH/FaceAlignment_WFLWds_ADNetwithSTARloss.git --branch WFLWds_ADNetwithSTARLoss
!pip install -r requirements.txt

In [ ]:
%cd /content
!git clone https://github.com/zhenglinzhou/star

/content
Cloning into 'star'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 72 (delta 8), reused 69 (delta 5), pack-reused 0
Receiving objects: 100% (72/72), 990.84 KiB | 7.86 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
%cd /content
import wandb
wandb.login(key="c8767797aae76cbcd389ff29929ace1ac3021161")
run = wandb.init()
artifact = run.use_artifact('doanngoccuong_nh/FaceAlignment_ADNetwithSTARloss/ADNetSTARLoss_bestmodel:v0', type='model')
artifact_dir = artifact.download()

# https://drive.usercontent.google.com/download?id=1aOx0wYEZUfBndYy_8IYszLPG_D2fhxrT&export=download&authuser=0
# https://drive.usercontent.google.com/download?id=1aOx0wYEZUfBndYy_8IYszLPG_D2fhxrT
# %cd /content/drive/MyDrive/colab
%cd /content
import gdown
gdown.download('https://drive.google.com/u/0/uc?id=1aOx0wYEZUfBndYy_8IYszLPG_D2fhxrT', 'WFLW_STARLoss_NME_4_02_FR_2_32_AUC_0_605.pkl')


In [ ]:
# %cd /content/drive/MyDrive/colab
!git clone https://github.com/italojs/facial-landmarks-recognition

- Thêm đường dẫn landmark vào file demo.py
- Thêm đường dẫn best model vào file demo.py
-

In [ ]:
!pip install gradio

In [ ]:
%cd /content/FaceAlignment_WFLWds_ADNetwithSTARloss
!python demo.py



- output ra 1 đống DEBUG và Figure(640x480) chỉ thấy như này, ko thấy ảnh ra gì cả, dù trong code có plt.show().
Nguyên nhân có thể là do ảnh kích thước to quá.

- Test ko nên chọn ảnh bị che 1 phần mặt


## 2.5 INFER - EVALUATION on BIG TESTSET

# 3.